In [1]:
import numpy as np
import pandas as pd
import os
import h5py
import matplotlib
import math
from matplotlib import pyplot as plt
import pickle
# %matplotlib inline
# matplotlib.style.use('ggplot')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Keras
# import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers

# Tensorflow

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# Custom imports
from physionet_processing import (fetch_h5data, spectrogram, 
                                  special_parameters, transformed_stats)

from physionet_generator import DataGenerator

FILENAME = "testing"

print('Tensorflow version:', tf.__version__)
# print('Keras version:', keras.__version__)

#Open hdf5 file, load the labels and define training/validation splits

# Data folder and hdf5 dataset file
data_root = os.path.normpath('.')
#data_root = os.path.normpath('/media/sf_vbshare/physionet_data/')
#data_root = os.path.normpath('/home/ubuntu/projects/csproject')
# hd_file = os.path.join(data_root, 'physio.h5')
hd_file = "/scratch/thurasx/ecg_project_2/cnn_ecg_keras/physio.h5"
label_file = "/scratch/thurasx/ecg_project_2/cnn_ecg_keras/REFERENCE-v3.csv"

# mac 
# hd_file = "/Users/macbookpro/Documents/physio.h5"
# label_file = "/Users/macbookpro/Documents/ecg_project_2/cnn_ecg_keras/REFERENCE-v3.csv"


# Open hdf5 file
h5file =  h5py.File(hd_file, 'r')




# Get a list of dataset names 
dataset_list = list(h5file.keys())

# Load the labels
label_df = pd.read_csv(label_file, header = None, names = ['name', 'label'])
# Filter the labels that are in the small demo set
label_df = label_df[label_df['name'].isin(dataset_list)]


# Encode labels to integer numbers
label_set = list(sorted(label_df.label.unique()))
encoder = LabelEncoder().fit(label_set)
label_set_codings = encoder.transform(label_set)
label_df = label_df.assign(encoded = encoder.transform(label_df.label))
labels = dict(zip(label_df.name, label_df.encoded))

# print(label_df)
encoded = label_df['encoded'].to_numpy()
a, b = np.unique(encoded, return_counts=True)
print(a)
print(b)
"""broad casting"""
l0 = label_df['name'].to_numpy()[encoded == 0]
l1 = label_df['name'].to_numpy()[encoded == 1]
l2 = label_df['name'].to_numpy()[encoded == 2]
l3 = label_df['name'].to_numpy()[encoded == 3]

print(len(l0),len(l1),len(l2),len(l3))
print(type(l2))

""" Train data """
train_data = np.array([])
random_count_train = len(l3)
train_data = np.concatenate((train_data,np.random.choice(l0, size=random_count_train, replace=False)), axis=0)
train_data = np.concatenate((train_data,np.random.choice(l1, size=random_count_train, replace=False)), axis=0)
train_data = np.concatenate((train_data,np.random.choice(l2, size=random_count_train, replace=False)), axis=0)
train_data = np.concatenate((train_data,l3), axis=0)
assert(len(train_data) == random_count_train * 4)
print(len(train_data))


""" Test data """
test_data = np.array([])
random_count_test = math.floor(len(l3) * 0.5)
print(random_count_test)
test_data = np.concatenate((test_data,np.random.choice(l0, size=random_count_test, replace=False)), axis=0)
test_data = np.concatenate((test_data,np.random.choice(l1, size=random_count_test, replace=False)), axis=0)
test_data = np.concatenate((test_data,np.random.choice(l2, size=random_count_test, replace=False)), axis=0)
test_data = np.concatenate((test_data,np.random.choice(l3, size=random_count_test, replace=False)), axis=0)
assert(len(test_data) == random_count_test * 4)
print(len(test_data), random_count_test * 4)

# Split the IDs in training and validation set
test_split = 0.30
idx = np.arange(label_df.shape[0])
id_train, id_val, _, _ = train_test_split(train_data, train_data, 
                                        test_size = test_split,
                                        shuffle = True,
                                        random_state = 123)

partition = { 'train' : id_train,
             'validation': id_val,
             'test' : test_data}


labels = dict(zip(label_df.name, label_df.encoded))
with open(f"/scratch/thurasx/ecg_project_2/cnn_ecg_keras/cnn_ecg_testing/cnn_small_{FILENAME}_testlabel.pcl", "wb") as f:
    pickle.dump(partition["test"], f)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6229962120961834603
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 49516445696
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13720036999912422177
physical_device_desc: "device: 0, name: Quadro RTX 8000, pci bus id: 0000:3b:00.0, compute capability: 7.5"
xla_global_id: 416903419
]
Tensorflow version: 2.7.0
[0 1 2 3]
[ 758 5076 2415  279]
758 5076 2415 279
<class 'numpy.ndarray'>
1116
139
556 556


2023-03-25 21:13:10.721071: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-25 21:13:14.126814: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 47222 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:3b:00.0, compute capability: 7.5
